In [3]:
# ps-29
%pip install pyspark

In [4]:
# import all necessary module
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [5]:
# create spark session
spark = SparkSession.builder.master('local').appName('pyspark-day-26').getOrCreate()
spark

1️⃣  Get employee name, project name order by firstname from "EmployeeDetail" and "ProjectDetail" for all employee even they have not assigned project.
2️⃣  Get employee name, project name order by firstname from "EmployeeDetail" and "ProjectDetail" for all employee if project is not assigned then display "-No Project Assigned".

Schema And Data:
================
# Input Data:
data = [
 [1, "Vikas", "Ahlawat", 600000.0, "2013-02-15 11:16:28.290", "IT", "Male"],
 [2, "nikita", "Jain", 530000.0, "2014-01-09 17:31:07.793", "HR", "Female"],
 [3, "Ashish", "Kumar", 1000000.0, "2014-01-09 10:05:07.793", "IT", "Male"],
 [4, "Nikhil", "Sharma", 480000.0, "2014-01-09 09:00:07.793", "HR", "Male"],
 [5, "anish", "kadian", 500000.0, "2014-01-09 09:31:07.793", "Payroll", "Male"],
]

# Create a schema for the DataFrame
schema = StructType([
 StructField("EmployeeID", IntegerType(), True),
 StructField("First_Name", StringType(), True),
 StructField("Last_Name", StringType(), True),
 StructField("Salary", DoubleType(), True),
 StructField("Joining_Date", StringType(), True),
 StructField("Department", StringType(), True),
 StructField("Gender", StringType(), True)
])

pro_schema = StructType([
StructField("Project_DetailID", IntegerType(), True),
StructField("Employee_DetailID", IntegerType(), True),
StructField("Project_Name", StringType(), True)
])

# Create the data as a list of tuples
pro_data = [
(1, 1, "Task Track"),
(2, 1, "CLP"),
(3, 1, "Survey Management"),
(4, 2, "HR Management"),
(5, 3, "Task Track"),
(6, 3, "GRS"),
(7, 3, "DDS"),
(8, 4, "HR Management"),
(9, 6, "GL Management")
]

In [6]:
# Input Data:
data = [
 [1, "Vikas", "Ahlawat", 600000.0, "2013-02-15 11:16:28.290", "IT", "Male"],
 [2, "nikita", "Jain", 530000.0, "2014-01-09 17:31:07.793", "HR", "Female"],
 [3, "Ashish", "Kumar", 1000000.0, "2014-01-09 10:05:07.793", "IT", "Male"],
 [4, "Nikhil", "Sharma", 480000.0, "2014-01-09 09:00:07.793", "HR", "Male"],
 [5, "anish", "kadian", 500000.0, "2014-01-09 09:31:07.793", "Payroll", "Male"],
]

# Create a schema for the DataFrame
schema = StructType([
 StructField("EmployeeID", IntegerType(), True),
 StructField("First_Name", StringType(), True),
 StructField("Last_Name", StringType(), True),
 StructField("Salary", DoubleType(), True),
 StructField("Joining_Date", StringType(), True),
 StructField("Department", StringType(), True),
 StructField("Gender", StringType(), True)
])

In [7]:
emp_df = spark.createDataFrame(schema = schema, data = data)
emp_df.show()

+----------+----------+---------+---------+--------------------+----------+------+
|EmployeeID|First_Name|Last_Name|   Salary|        Joining_Date|Department|Gender|
+----------+----------+---------+---------+--------------------+----------+------+
|         1|     Vikas|  Ahlawat| 600000.0|2013-02-15 11:16:...|        IT|  Male|
|         2|    nikita|     Jain| 530000.0|2014-01-09 17:31:...|        HR|Female|
|         3|    Ashish|    Kumar|1000000.0|2014-01-09 10:05:...|        IT|  Male|
|         4|    Nikhil|   Sharma| 480000.0|2014-01-09 09:00:...|        HR|  Male|
|         5|     anish|   kadian| 500000.0|2014-01-09 09:31:...|   Payroll|  Male|
+----------+----------+---------+---------+--------------------+----------+------+



In [8]:
pro_schema = StructType([
StructField("Project_DetailID", IntegerType(), True),
StructField("Employee_DetailID", IntegerType(), True),
StructField("Project_Name", StringType(), True)
])

# Create the data as a list of tuples
pro_data = [
(1, 1, "Task Track"),
(2, 1, "CLP"),
(3, 1, "Survey Management"),
(4, 2, "HR Management"),
(5, 3, "Task Track"),
(6, 3, "GRS"),
(7, 3, "DDS"),
(8, 4, "HR Management"),
(9, 6, "GL Management")
]

In [9]:
# create df for project
pro_df = spark.createDataFrame(schema = pro_schema, data = pro_data)
pro_df.show()

+----------------+-----------------+-----------------+
|Project_DetailID|Employee_DetailID|     Project_Name|
+----------------+-----------------+-----------------+
|               1|                1|       Task Track|
|               2|                1|              CLP|
|               3|                1|Survey Management|
|               4|                2|    HR Management|
|               5|                3|       Task Track|
|               6|                3|              GRS|
|               7|                3|              DDS|
|               8|                4|    HR Management|
|               9|                6|    GL Management|
+----------------+-----------------+-----------------+



In [10]:
emp_df.join(pro_df, emp_df['EmployeeID'] == pro_df['Employee_DetailID'],'left')\
      .select('First_Name','Project_Name')\
      .orderBy(lower(col('First_Name')))\
      .show()

+----------+-----------------+
|First_Name|     Project_Name|
+----------+-----------------+
|     anish|             NULL|
|    Ashish|              DDS|
|    Ashish|              GRS|
|    Ashish|       Task Track|
|    Nikhil|    HR Management|
|    nikita|    HR Management|
|     Vikas|Survey Management|
|     Vikas|              CLP|
|     Vikas|       Task Track|
+----------+-----------------+



In [11]:
left_df = emp_df.join(pro_df, emp_df['EmployeeID'] == pro_df['Employee_DetailID'],'left')\
      .select('EmployeeID','First_Name','Last_Name','Project_Name')\
      .orderBy(lower(col('First_Name')))

left_df.withColumn('Project_Name',coalesce(col('Project_Name'), lit('- No Project'))).show()

+----------+----------+---------+-----------------+
|EmployeeID|First_Name|Last_Name|     Project_Name|
+----------+----------+---------+-----------------+
|         5|     anish|   kadian|     - No Project|
|         3|    Ashish|    Kumar|              DDS|
|         3|    Ashish|    Kumar|              GRS|
|         3|    Ashish|    Kumar|       Task Track|
|         4|    Nikhil|   Sharma|    HR Management|
|         2|    nikita|     Jain|    HR Management|
|         1|     Vikas|  Ahlawat|Survey Management|
|         1|     Vikas|  Ahlawat|              CLP|
|         1|     Vikas|  Ahlawat|       Task Track|
+----------+----------+---------+-----------------+

